# **Supermarket Transaction Analysis in SQL**  
**Date:** March 14, 2023 · **Dataset:** [Polish Supermarket Transactions](https://www.mdpi.com/2306-5729/4/2/67/htm)  

## **Overview**  
Retailers rely on POS (point-of-sale) transaction data to optimize sales, staffing, and operations. This analysis uses SQL to answer key questions:  
- Payment method preference (card vs cash)  
- Spending behavior by payment method  
- Impact of Poland’s Sunday trading ban  
- Daily and weekly sales trends  
- Staffing optimization for cost savings  

---

## **Task 1 – Data Exploration**  
**Objective:** Understand the database structure and key fields.  

In [3]:
SELECT * FROM 'pos_operator_logs.csv' LIMIT 100

,id,Workstation_Group_ID,Workstation_ID,begin_date_time,operator_id
0,1,8,19,2019-02-13 05:37:55+00:00,269
1,2,8,18,2019-02-13 05:37:55+00:00,268
2,3,8,17,2019-02-13 05:38:43+00:00,267
3,4,1,4,2019-02-13 07:01:26+00:00,332
4,5,1,7,2019-02-13 07:01:57+00:00,10
...,...,...,...,...,...
95,96,1,20,2019-02-13 10:13:59+00:00,328
96,97,1,21,2019-02-13 10:17:43+00:00,338
97,98,1,20,2019-02-13 10:24:06+00:00,328
98,99,1,20,2019-02-13 10:24:24+00:00,328


In [4]:
SELECT * FROM 'pos_transactions.csv' LIMIT 100

,id,WorkstationGroupID,begin_date_time,end_date_time,OperatorID,basket_size,t_cash,t_card,amount
0,1,1,2017-12-07 14:23:23+00:00,2017-12-07 14:24:36+00:00,101,23,True,False,112.71
1,2,1,2017-12-07 14:25:09+00:00,2017-12-07 14:27:00+00:00,101,29,True,False,54.76
2,3,1,2017-12-07 14:27:28+00:00,2017-12-07 14:27:48+00:00,101,3,True,False,14.77
3,4,1,2017-12-07 14:28:04+00:00,2017-12-07 14:28:29+00:00,101,12,True,False,37.88
4,5,1,2017-12-07 14:29:40+00:00,2017-12-07 14:30:32+00:00,101,7,True,False,115.34
...,...,...,...,...,...,...,...,...,...
95,96,1,2017-12-07 17:07:48+00:00,2017-12-07 17:08:39+00:00,101,9,True,False,32.54
96,97,1,2017-12-07 17:10:51+00:00,2017-12-07 17:11:33+00:00,101,9,True,False,26.59
97,98,1,2017-12-07 17:11:47+00:00,2017-12-07 17:12:37+00:00,101,20,True,False,104.82
98,99,1,2017-12-07 17:12:51+00:00,2017-12-07 17:13:58+00:00,101,12,False,True,67.62


**Insight:** Confirmed fields for payment type (`t_cash`, `t_card`), basket size, transaction amount, operator logs, and timestamps.  

## **Task 2 – Payment Method Popularity**  
**Question:** Do more people pay by card or by cash?  

## Payment Method Popularity (Card vs Cash)

**Question**  
Are shoppers using card or cash more often?

**What the data shows**  
- Card transactions are slightly **higher** than cash.  
- The gap is consistent across the sample — not a one-week spike.

**So what? (Business impact)**  
- Prioritize **card terminal capacity** and **terminal uptime** over adding cash lanes.  
- Emphasize **card-linked promos** (points, instant discounts) — they reach the bigger payment cohort.  
- Expect **faster lanes** with card due to no cash handling; schedule staff accordingly.

**Next actions**  
- Track the **card share trend** monthly to catch shifts (e.g., tourist season, economic stress).  
- A/B test a **card-only promo** vs **cash discount** week and measure ticket size lift.


In [5]:
SELECT 

COUNT(CASE WHEN t_cash THEN 1 END) AS cash_transactions,
COUNT(CASE WHEN t_card THEN 1 END) AS card_transactions
		
FROM 'pos_transactions.csv'

,cash_transactions,card_transactions
0,84487,78246


## **Task 3 – Spending by Payment Method**  
**Question:** Which method sees higher spend per transaction?  

## Average Spend by Method

**Question**  
Do people **spend more per transaction** with card or cash?

**What the data shows**  
- **Card** average ticket > **Cash** average ticket.

**So what? (Business impact)**  
- Card users are your **higher-value** customers; they are the best audience for **basket-expansion** nudges (cross-sell at checkout, buy-more-save-more).  
- Cash lanes are still important, but ROI on **impulse displays** and **last-mile upsells** is likely higher on card lanes.

**Next actions**  
- Run a **category attach-rate** analysis split by payment type (e.g., drinks attached to ready-meals).  
- Pilot **card-linked offers** (e.g., “+10% points if you add dessert”), then compare lift vs control.

In [6]:
SELECT 

AVG(CASE WHEN t_cash AND NOT t_card THEN amount END) AS cash_transactions,
AVG(CASE WHEN t_card AND NOT t_cash THEN amount END) AS card_transactions

FROM 'pos_transactions.csv'

,cash_transactions,card_transactions
0,58.718519,86.83937


In [7]:
SELECT AVG(amount)
FROM 'pos_transactions.csv'
WHERE t_cash AND NOT t_card

,avg(amount)
0,58.718519


In [8]:
SELECT AVG(amount)
FROM 'pos_transactions.csv'
WHERE t_card AND NOT t_cash

,avg(amount)
0,86.83937


## **Task 4 – Sunday Trading Impact**  
**Context:** In 2019, only certain Sundays allowed trading.  
**Goal:** Identify working vs non-working Sundays by week.  

## Sunday Trading Rules & Shopper Behavior

**Question**  
What’s happening on **working vs non-working Sundays**?

**What the data shows**  
- Working Sundays fall in **Weeks 8 & 13**. Non-working in **Weeks 7 & 14**.  
- Weekly sales in **Week 8 (WS)** ≈ **Week 14 (NWS)** (≈ PLN 1.7M), suggesting **pre-Sunday pull-forward** when stores are closed.

**So what? (Business impact)**  
- Closing Sundays does **not** reduce weekly revenue; shoppers shift to **Fri/Sat**.  
- Opportunity to **extend Friday/Saturday hours** and staff more lanes those days instead of Sunday.

**Next actions**  
- Create **“Sunday-closed packs”** (bulk, ready-to-eat) promoted **Fri/Sat**.  
- Monitor **OOS rates** on Sat evenings to right-size inventory before non-working Sundays.

In [9]:
SELECT 
DATE_PART('week', end_date_time::DATE) as week_num,
end_date_time::DATE AS end_date
FROM 'pos_transactions.csv' 
WHERE EXTRACT(YEAR FROM end_date_time) >= '2019' 
    AND end_date IN ('2019-02-24','2019-03-31','2019-02-17','2019-04-07')
GROUP BY end_date


,week_num,end_date
0,13,2019-03-31 00:00:00+00:00
1,8,2019-02-24 00:00:00+00:00


**Result:**  
- Working Sundays: Weeks 8 & 13  
- Non-working Sundays: Weeks 7 & 14 

## **Task 5 – Daily Trends (2019)**  
**Goal:** Identify busiest days and spending patterns.  


## Daily Trends (2019)

**Question**  
Which days are busiest and most valuable?

**What the data shows**  
- **Thu–Sat**: ~**+25%** more transactions and ~**+40%** higher sales vs **Mon–Wed**.  
- **Sundays**: lowest traffic (as expected).  
- Average ticket and basket size also tend to be **higher** late-week.

**So what? (Business impact)**  
- **Staffing**: Load more cashiers/self-checkout attendants **Thu–Sat**; reduce Mon–Tue.  
- **Promotions**: Use **Mon–Tue** to pull traffic (e.g., “early-week value combos”); protect margins **Fri–Sat** (less discounting needed).

**Next actions**  
- Add a **power-hour chart** (hour of day × day of week) to refine shift starts.  
- Re-slot end-caps to **late-week impulse categories** (snacks, drinks, desserts).


In [10]:
SELECT 
DATE_PART('week', end_date_time::DATE) as week_num,
end_date_time::DATE AS end_date,
COUNT(id) as total_transactions,
SUM(amount) as total_sales,
AVG(amount) as avg_sale_amount

FROM 'pos_transactions.csv'
WHERE EXTRACT (YEAR FROM (end_date)) >= '2019'
GROUP BY end_date ORDER BY week_num

,week_num,end_date,total_transactions,total_sales,avg_sale_amount
0,7,2019-02-15 00:00:00+00:00,4031,319112.44,79.164584
1,7,2019-02-14 00:00:00+00:00,4404,302242.45,68.629076
2,7,2019-02-13 00:00:00+00:00,3163,189112.49,59.788963
3,7,2019-02-16 00:00:00+00:00,4669,405580.79,86.866736
4,8,2019-02-22 00:00:00+00:00,4411,330428.78,74.910175
5,8,2019-02-18 00:00:00+00:00,3270,193133.29,59.062168
6,8,2019-02-21 00:00:00+00:00,4368,298612.75,68.363725
7,8,2019-02-19 00:00:00+00:00,3025,170242.42,56.278486
8,8,2019-02-24 00:00:00+00:00,2258,143025.41,63.341634
9,8,2019-02-23 00:00:00+00:00,4367,369412.97,84.591933


Build on your analysis adding the average and median `basket_size` per day in 2019. 



In [11]:
SELECT 
DATE_PART('week', end_date_time::DATE) as week_num,
end_date_time::DATE AS end_date,
COUNT(id) as total_transactions,
SUM(amount) as total_sales,
AVG(amount) as avg_sale_amount,
PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY amount)  as median_sale_amount,
AVG(basket_size) as avg_basket_size,
PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY basket_size)  as median_basket_size

FROM 'pos_transactions.csv'
WHERE EXTRACT(YEAR FROM end_date_time) >= '2019'
GROUP BY (end_date) ORDER BY week_num 

,week_num,end_date,total_transactions,total_sales,avg_sale_amount,median_sale_amount,avg_basket_size,median_basket_size
0,7,2019-02-15 00:00:00+00:00,4031,319112.44,79.164584,50.900,17.184818,12.0
1,7,2019-02-16 00:00:00+00:00,4669,405580.79,86.866736,57.600,19.733348,14.0
2,7,2019-02-13 00:00:00+00:00,3163,189112.49,59.788963,37.580,12.915903,9.0
3,7,2019-02-14 00:00:00+00:00,4404,302242.45,68.629076,46.285,14.172116,10.0
4,8,2019-02-23 00:00:00+00:00,4367,369412.97,84.591933,56.370,19.813144,14.0
5,8,2019-02-19 00:00:00+00:00,3025,170242.42,56.278486,37.530,13.432727,9.0
6,8,2019-02-20 00:00:00+00:00,3045,181630.10,59.648637,36.560,13.594417,9.0
7,8,2019-02-21 00:00:00+00:00,4368,298612.75,68.363725,45.865,16.488324,12.0
8,8,2019-02-18 00:00:00+00:00,3270,193133.29,59.062168,37.420,13.810092,10.0
9,8,2019-02-22 00:00:00+00:00,4411,330428.78,74.910175,49.060,17.736568,12.0


## **Task 6 – Weekly Revenue: WS vs NWS**  
**Goal:** Compare weekly sales between working and non-working Sunday weeks.  



## Weekly Revenue – WS vs NWS

**Question**  
Does opening on Sunday change the **weekly** total?

**What the data shows**  
- **Week 8 (WS)** ≈ **Week 14 (NWS)** in total sales (~PLN 1.7M).  
- Demand shifts, not disappears: shoppers buy earlier when Sunday is closed.

**So what? (Business impact)**  
- If competitors also close, you can **save costs** by closing Sunday with **no revenue penalty**.  
- Concentrate **ops & replenishment** on **Fri–Sat** to capture the shift cleanly.

**Next actions**  
- Model **labor costs saved** vs **incremental Sat overtime**.  
- Track **customer wait times** and **basket abandonment** Sat evening (add mobile checkout if needed).

In [12]:
SELECT 
DATE_PART('week', end_date_time::DATE) as week_num,
SUM(amount)
FROM 'pos_transactions.csv'

WHERE EXTRACT(YEAR FROM end_date_time) >= '2019' AND week_num IN ('8','14')

GROUP BY week_num


,week_num,sum(amount)
0,8,1686485.72
1,14,1690989.87


## **Task 7 – Staffing Costs on Sundays**  
**Goal:** Count operators on working Sundays.  

## Staffing on Sundays

**Question**  
How many operators are we scheduling on working Sundays?

**What the data shows**  
- **Feb 24 (WS): 18 operators**, **Mar 31 (WS): 20 operators**.

**So what? (Business impact)**  
- If Sundays close, you can **reassign ~18–20 operators** to **Fri/Sat** peaks or reduce total hours.  
- Combine with energy & security savings to estimate **full Sunday cost avoidance**.

**Next actions**  
- Calculate **cost per Sunday** = (wages + energy + security + shrink) − (lost sales * margin delta).  
- If Sunday stays open, **shorten hours** and move staff to **self-checkout coverage** where throughput is higher.

In [13]:
SELECT 
(begin_date_time::DATE) as working_day,
COUNT(DISTINCT(operator_id)),
DATE_PART('week',(begin_date_time::DATE)) as week_num

FROM 'pos_operator_logs.csv'

WHERE EXTRACT(YEAR FROM begin_date_time) >= '2019'

GROUP BY working_day

,working_day,count(DISTINCT operator_id),week_num
0,2019-02-13 00:00:00+00:00,20,7
1,2019-02-14 00:00:00+00:00,26,7
2,2019-02-25 00:00:00+00:00,20,9
3,2019-02-26 00:00:00+00:00,20,9
4,2019-04-03 00:00:00+00:00,21,14
5,2019-04-08 00:00:00+00:00,21,15
6,2019-02-21 00:00:00+00:00,27,8
7,2019-02-15 00:00:00+00:00,24,7
8,2019-03-31 00:00:00+00:00,20,13
9,2019-04-02 00:00:00+00:00,21,14


**Result:**  
- 24 Feb: 18 operators  
- 31 Mar: 20 operators  

**Implication:** Closing Sundays could save labor costs without harming revenue, assuming competitive behavior matches.  

---

## **Key Insights**  
- **Card dominates** both in transaction count and spend.  
- **Sunday closure** has no negative impact on weekly revenue.  
- **Peak days** are Thurs–Sat — ideal for staff allocation.  
- **Cost savings** possible through reduced Sunday operations.  